In [ ]:
from pytential.sympy_pytential import sympy_pytential
from pytential.reduce import min_pytential, pyt_minimizer

In [ ]:
fa = sympy_pytential.from_properties({'mu0':[2,0]}, {'T':.300}, suffix='a')
fb = sympy_pytential.from_properties({'mu0':[0,2]}, {'T':.300}, suffix='b')

Make a collected function fa+fb with all the variables, and add constraints that the concentrations must sum to ca and cb

In [ ]:
f = fa+fb
f = sympy_pytential.sum_extensive_variables(f, ['c0', 'c1'])
print(f)

In [ ]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator, RBFInterpolator, NearestNDInterpolator

In [ ]:
x0 = np.array([[0.5, 0.5, 0.5,0.24999999,0.25000001,0.5,  0.25000001, 0.24999999]])
y0 = [[0.5, 0.5, 0.5, 0.5]]
NearestNDInterpolator(y0, x0)([0.5, 0.5, 0.2, 0.4])

In [ ]:
f2 = min_pytential(f, vars_out=['c0', 'c1', 'Va', 'Vb'])
#f2([[.5,.5,.5,.5],[.4,.6,.5,.5],[.6,.4,.5,.5],[.6,.4,.5,.6]]) 

Visualization

In [ ]:
from numpy import linspace, vectorize, meshgrid
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

c0 = linspace(0.001, 0.999, 3)
va = linspace(0.001, 0.999, 3)
C0, Va = meshgrid(c0, va)

args_matrix = np.array([C0.flatten(), (1 - C0).flatten(), Va.flatten(), (1 - Va).flatten()]).T
print(args_matrix)
# Compute Z values using the fa function
F_flat = f2(args_matrix)
print(F_flat)
F = F_flat.reshape(C0.shape)

In [40]:
import sympy as sp
x = sp.symbols('x')
f = sp.log(x)
print(f)
fcn = sp.lambdify(x, f, 'scipy')
fcn(1e-6)

fr = replace_log_with_log1p(f)
print(fr)
fcnr = sp.lambdify(x, fr, 'scipy')
fcnr(0)

log(x)
log1p(x - 1)


np.float64(-inf)

In [33]:
def replace_log_with_log1p(expr):
    from sympy.codegen.cfunctions import log1p
    return expr.replace(sp.log, lambda arg: log1p(arg - 1))

In [25]:
fr = replace_log_with_log1p(f)
print(fr)


log1p(x - 1)


In [29]:
fcn = sp.lambdify(x, f, 'scipy')
print(f)
f(.4)

log(x)


TypeError: 'log' object is not callable

In [ ]:
#fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'xy'}]])
fig = go.Figure()

# Add the 3D surface plot for f
fig.add_trace(go.Surface(
    z=F, 
    x=c0, 
    y=va, 
    colorscale='Viridis',
    contours={
        "x": {"show": True, "color": "white"},
        "y": {"show": True, "color": "white"},
        "z": {"show": False, "color": "white"}
    }
))

fig.add_trace(go.Scatter3d(x=c0, y=[1]*len(c0), z=fa(c0a=c0,c1a=1-c0,Va=1), mode='lines', name='fa', line=dict(color='red', width=4)))
fig.add_trace(go.Scatter3d(x=c0, y=[0]*len(c0), z=fb(c0b=c0,c1b=1-c0,Vb=1), mode='lines', name='fb', line=dict(color='blue', width = 4)))

# # Add the 2D plot for f_reduced
# fig.add_trace(go.Scatter(x=c0, y=fa(c0a=c0,c1a=1-c0,Va=1), mode='lines', name='fa'), row=1, col=2)
# fig.add_trace(go.Scatter(x=c0, y=fb(c0b=c0,c1b=1-c0,Vb=1), mode='lines', name='fb'), row=1, col=2)
# fig.add_trace(go.Scatter(x=c0, y=f2(c0=c0,c1=1-c0,Va=1, Vb=0), mode='lines', name='fb'), row=1, col=2)

# Update layout for the entire figure
fig.update_layout(
    title='3D Surface Plot of f and 2D Plot of f_reduced',
    scene=dict(
        xaxis_title='c0',
        yaxis_title='Va',
        zaxis_title='f',
        aspectratio = dict(x=2, y=2, z=1),
        camera=dict(projection=dict(type="orthographic"))
    ),
)
fig.show()

In [ ]:
f3 = min_pytential(f, vars_out=['Va', 'Vb'])

In [ ]:

f3(Va=.5,Vb=.5)